In [103]:
import pandas as pd
import numpy as np
import rapidfuzz

In [104]:
state = 'RI'
path = r"C:\Users\david\OneDrive\Documents\GitHub\SPEOC-pt-1\Data\Post1790\RI\T653_Rhode_Island_CD.xlsx"
occs_and_debt = pd.read_excel(path,
                             header=11, usecols='J, K, M, Z, AA, AN, AO') #occupations plus continental debt for conneticut
print(occs_and_debt)

    Unnamed: 9 Occupation  Cents  Dollars.1  Cents.1  Dollars.2  Cents.2
0           RI        NaN   86.0      295.0     43.0      216.0      NaN
1           RI    Esquire   88.0      281.0     94.0      482.0     18.0
2           RI    Esquire   28.0      432.0     14.0      306.0      NaN
3           RI     Farmer   33.0      666.0     66.0      480.0      NaN
4           RI    Esquire   50.0       14.0     74.0       17.0     13.0
..         ...        ...    ...        ...      ...        ...      ...
657        NaN        NaN    NaN       18.0     78.0        NaN      NaN
658        NaN        NaN    NaN        NaN      NaN        NaN      NaN
659        NaN        NaN    NaN       22.0     56.0        NaN      NaN
660        NaN        NaN    NaN       17.0      5.0        NaN      NaN
661        NaN        NaN    NaN       10.0     10.0        NaN      NaN

[662 rows x 7 columns]


In [105]:
occs_and_debt = occs_and_debt.rename(columns={'Occupation.1':'Occupation'})
occs_and_debt = occs_and_debt.dropna(subset=['Occupation'])
print(occs_and_debt)

    Unnamed: 9   Occupation  Cents  Dollars.1  Cents.1  Dollars.2  Cents.2
1           RI      Esquire   88.0      281.0     94.0      482.0     18.0
2           RI      Esquire   28.0      432.0     14.0      306.0      NaN
3           RI       Farmer   33.0      666.0     66.0      480.0      NaN
4           RI      Esquire   50.0       14.0     74.0       17.0     13.0
5           RI       Yeoman    NaN       56.0      NaN       76.0     46.0
..         ...          ...    ...        ...      ...        ...      ...
604        NaN  Providence    60.0        NaN      NaN       82.0     62.0
608        NaN  Providence    41.0       89.0     20.0       96.0      NaN
616         RI      Esquire   90.0       29.0     95.0       70.0      8.0
617         RI       Yeoman   44.0       14.0     22.0       29.0     70.0
632         RI     Merchant   44.0       71.0     22.0      132.0     43.0

[518 rows x 7 columns]


In [106]:
#exclude non states 
occs_and_debt = occs_and_debt.rename(columns={'Unnamed: 9':'State'})
occs_and_debt = occs_and_debt[occs_and_debt['State'].str.contains(state) == True].drop('State', axis=1)

In [107]:
occurences_dict = occs_and_debt['Occupation'].value_counts().to_dict()
print(occurences_dict)

{'Esquire': 129, 'Merchant': 92, 'Yeoman': 78, 'Mariner': 24, 'Widow': 23, 'Gentleman': 10, 'Innholder': 8, 'Physician': 7, 'Trader': 5, 'Taylor': 4, 'Distiller': 3, 'Spinster': 3, 'Mason': 3, 'Blacksmith': 3, 'Auctioneers': 3, 'Cooper': 3, 'Treasurer of the State of Rhode Island Of Newport': 2, 'Shipwright': 2, 'Hatter': 2, 'Reverend': 2, 'Labourer': 2, 'Broker': 2, 'Treasurer of the State of Rhode Island Of Newport Esquire': 1, 'Administratrix to the Estate of Joshua Brown Esquire Deceased': 1, 'Baker': 1, 'Treasurer of Scituate': 1, 'Deacon': 1, 'Tresurer of the Catholic Society at Bristol': 1, 'treasurer of the Town of Providence': 1, 'Traders In Company ': 1, 'Goldsmith': 1, 'Blockmaker': 1, 'in behalf of the Proprietors of Tiverton': 1, 'Merchants': 1, 'Treasurer to the Charitable Baptist Society in Providence': 1, ' Doctor': 1, 'Extors to the Late Nicholas Brown Esq Deceased': 1, 'Glazier': 1, 'Chaise Maker': 1, 'Esquier': 1, 'Treasurer of the Congregational Benevolent Society i

In [108]:
totals = occs_and_debt.groupby('Occupation')[['Dollars', 'Dollars.1', 'Dollars.2', 'Cents', 'Cents.1', 'Cents.2']].apply(sum)
totals = totals.reset_index()
print(totals[['Occupation', 'Dollars', 'Cents', 'Dollars.1', 'Cents.1']].to_string())

KeyError: "Columns not found: 'Dollars'"

In [ ]:
occs_debt_wo_dups = totals.copy()
#use fuzzy string matching to remove any similar names like merchant and merchents. does not take into account longer titles.
occ_list = occs_debt_wo_dups['Occupation'].tolist()

for occ in occ_list:
    for occ_2 in occ_list:
        if occ != occ_2:
            if rapidfuzz.fuzz.ratio(occ, occ_2) >= 80 and occ != occ_2:
                #transfer amount in row that will be deleted to other row
                row_w_occ_2 = occs_debt_wo_dups.loc[occs_debt_wo_dups['Occupation'] == occ_2] #locate row that contains occupation soon to be deleted
                dollars_in_occ_2 = row_w_occ_2[['Dollars', 'Dollars.1', 'Dollars.2']].sum(axis=1).values #get total dollars
                cents_in_occ_2 = row_w_occ_2[['Cents', 'Cents.1', 'Cents.2']].sum(axis=1).values #get total cents
                dollars_in_occ_2 = dollars_in_occ_2[0] #stored in array. need to retrieve value
                cents_in_occ_2 = cents_in_occ_2[0] #stored in array. need to retrieve value  
                occs_debt_wo_dups.loc[(occs_debt_wo_dups.Occupation == occ), 'Dollars'] += dollars_in_occ_2 #add dollars to occupation 1
                occs_debt_wo_dups.loc[(occs_debt_wo_dups.Occupation == occ), 'Cents'] += cents_in_occ_2 #add cents to occupation 1
                 
                #remove value from occurences dictionary and add one to other occupation
                occurences_dict[occs_debt_wo_dups.loc[(occs_debt_wo_dups.Occupation == occ), 'Occupation'].values[0]] += occurences_dict[occs_debt_wo_dups.loc[(occs_debt_wo_dups.Occupation == occ_2), 'Occupation'].values[0]] 
                occurences_dict.pop(occ_2)
                
                #remove row
                occs_debt_wo_dups.drop(occs_debt_wo_dups[occs_debt_wo_dups['Occupation'] == occ_2].index, inplace=True) 
                occ_list.remove(occ_2)

occs_debt_wo_dups = occs_debt_wo_dups.reset_index(drop=True) #reset indexes 0-65 

#take the first two words of every job if applicable and remove any duplicate titles. handles longer titles 
two_word_occs = {}
#retrieve all two worded occupations
for occ in occ_list:
    if len(occ.split()) >= 2: #only account for words that are actually more than two words long
        occ_two_word = occ.split()[0] + " " + occ.split()[1] #get first two words
        two_word_occs[occ_list.index(occ)] = occ_two_word #append both words to dict

#retrieve any duplicates now 
flipped = {} 
for key, value in two_word_occs.items():
    flipped.setdefault(value, list()).append(key) #flip keys with values

keys_w_dups = [key for key, values in flipped.items() if len(values) > 1] #retrieve keys that have multiple values. store them in a list

print(occurences_dict)

In [ ]:
print(occs_debt_wo_dups.to_string())

In [ ]:
print(flipped)
print("\n\n")
print(keys_w_dups)

In [ ]:
print(occurences_dict)

for occ in keys_w_dups:
    i = 0
    while i < len(flipped[occ]):
        index = flipped[occ][i]
        if len(flipped[occ]) > 1: #make sure to keep one index in the list. don't want to delete ALL occurences
            duplicate_row = occs_debt_wo_dups.loc[[index]] #return row that has index
            dollars_in_dup_row = duplicate_row[['Dollars', 'Dollars.1', 'Dollars.2']].sum(axis=1).values #sum all dollars in row                
            cents_in_dup_row = duplicate_row[['Cents', 'Cents.1', 'Cents.2']].sum(axis=1).values #return total cents in row
            dollars_dup_row = dollars_in_dup_row[0] #stored in array. need to retrieve value
            cents_dup_row = cents_in_dup_row[0] #stored in array. need to retrieve value

            last_index = flipped[occ][len(flipped[occ]) - 1] #get last value in list. we will add dollars and cents here
                        
            occs_debt_wo_dups.loc[last_index, 'Dollars'] += dollars_dup_row #add dollars from one occupation to the other
            occs_debt_wo_dups.loc[last_index, 'Cents'] += cents_dup_row #add cents from one occupation to the other
            
            print(occs_debt_wo_dups.loc[last_index, 'Occupation'])
            occurences_dict[occs_debt_wo_dups.loc[last_index, 'Occupation']] += occurences_dict[occs_debt_wo_dups.loc[index, 'Occupation']] 
            occurences_dict.pop(occs_debt_wo_dups.loc[index, 'Occupation'])
            
            print("Length: " + str(len(flipped[occ])))
            print("Last Index: " + str(last_index) + " Index: " + str(index))
            print(flipped[occ])
            print("\n\n")
                        
            occs_debt_wo_dups.drop(index, inplace=True) #remove duplicate occupation now
            flipped[occ].remove(index) #remove from list
            
            #print(occurences_dict)
            #print("\n\n")
        else:
            i += 1

occs_debt_wo_dups = occs_debt_wo_dups.reset_index(drop=True) #reset indexes 0-...

In [ ]:
print(occs_debt_wo_dups[['Occupation', 'Dollars', 'Cents', 'Dollars.1', 'Cents.1']].to_string())

In [ ]:
print(flipped)

In [ ]:
print(occurences_dict)

In [ ]:
exceptions = ['General Treasurer of the state of Rhode Island and his suceessors in said office', 'in behalf of the Proprietors of Tiverton']

In [ ]:
#get the first word from every occupation and store any duplicates in a dictionary
occ_list = occs_debt_wo_dups['Occupation'].to_list()
occupations = []
one_word_occs = {}
flipped_one_w = {}

for occ in occ_list:
    if occ not in exceptions:
        first_word = occ.split()[0]
        one_word_occs[occ_list.index(occ)] = first_word 
        
#flip dictionary
for key, value in one_word_occs.items():
    flipped_one_w.setdefault(value, list()).append(key) #flip keys with values

In [ ]:
#use fuzzy string matching 
flipped_list = list(flipped_one_w) #convert to list. needed in order to iterate through
for occ in flipped_list:
    for occ_2 in flipped_list:
        if occ_2 != occ and rapidfuzz.fuzz.ratio(occ, occ_2) >= 80:
            flipped_one_w[occ].extend(flipped_one_w[occ_2]) #combine lists
            flipped_one_w.pop(occ_2) #remove second occurence of the same occupation 
            flipped_list.remove(occ_2) #remove from list 
            
keys_w_dups = [key for key, values in flipped_one_w.items() if len(values) > 1] #retrieve keys that have multiple values. store them in a list
         
print(flipped_one_w)
print("\n\n")
print(keys_w_dups)

In [ ]:
#remove duplicates
for occ in keys_w_dups:
    i = 0
    while i < len(flipped_one_w[occ]):
        if len(flipped_one_w[occ]) > 1:
            
            index = flipped_one_w[occ][i]
            last_index = flipped_one_w[occ][len(flipped_one_w[occ]) - 1] #get last value in list. we will add dollars and cents here
            duplicate_row = occs_debt_wo_dups.loc[[index]] #returns row that has index
            dollars_in_dup_row = duplicate_row[['Dollars', 'Dollars.1', 'Dollars.2']].sum(axis=1).values[0]
            cents_in_dup_row = duplicate_row[['Cents', 'Cents.1', 'Cents.2']].sum(axis=1).values[0]
            
            occs_debt_wo_dups.loc[last_index, 'Dollars'] += dollars_in_dup_row
            occs_debt_wo_dups.loc[last_index, 'Cents'] += cents_dup_row
            
            occurences_dict[occs_debt_wo_dups.loc[last_index, 'Occupation']] += occurences_dict[occs_debt_wo_dups.loc[index, 'Occupation']] 
            occurences_dict.pop(occs_debt_wo_dups.loc[index, 'Occupation'])

            occs_debt_wo_dups.drop(index, inplace=True)
            flipped_one_w[occ].remove(index)
            
        else:
            i += 1

occs_debt_wo_dups = occs_debt_wo_dups.reset_index(drop=True) #reset indexes 0-...

print(occs_debt_wo_dups[['Occupation', 'Dollars', 'Cents', 'Dollars.1', 'Cents.1']].to_string())

In [ ]:
occs_debt_wo_dups[['Cents', 'Cents.1', 'Cents.2']] = occs_debt_wo_dups[['Cents', 'Cents.1', 'Cents.2']].div(100)
print(occs_debt_wo_dups.to_string())

In [ ]:
occs_debt_wo_dups['Total'] = occs_debt_wo_dups[['Dollars', 'Dollars.1', 'Dollars.2', 'Cents', 'Cents.1', 'Cents.2']].sum(axis=1) #add totals to new column
print(occs_debt_wo_dups[['Occupation', 'Dollars', 'Cents', 'Dollars.1', 'Total']].to_string())

In [ ]:
occs_debt_wo_dups['# of People'] = occs_debt_wo_dups['Occupation'].map(occurences_dict) #add occurences to table
print(occs_debt_wo_dups[['Occupation', '# of People']])

In [ ]:
occs_debt_wo_dups['Average'] = occs_debt_wo_dups['Total'] / occs_debt_wo_dups['# of People'] #calculate average and add to new column
print(occs_debt_wo_dups[['Occupation', 'Total', '# of People', 'Average']].to_string())

In [ ]:
#convert to one word
exceptions = ['General Treasurer of the state of Rhode Island and his suceessors in said office', 'in behalf of the Proprietors of Tiverton']
def first_word_only(occ):
    if 'General Treasurer' in occ:
        return 'General Treasurer of Rhode Island'
    if 'treasurer' in occ:
        return 'Treasurer'
    if occ not in exceptions:
        return occ.split()[0]
    else:
        return occ
    
occs_debt_wo_dups['Occupation'] = occs_debt_wo_dups['Occupation'].apply(first_word_only)
print(occs_debt_wo_dups[['Occupation', 'Total', '# of People', 'Average']].to_string())

In [ ]:
total = occs_debt_wo_dups['Total'].sum()
occs_debt_wo_dups['% of Total Debt'] = round((occs_debt_wo_dups['Total'] / total) * 100, 3)
print(occs_debt_wo_dups[['Occupation', 'Total', '# of People', 'Average', '% of Total Debt']].to_string())

In [ ]:
#final total debt held by people w/ no job listed
occs_and_debt = pd.read_excel(path,
                             header=11, usecols='J, K, L, M, Z, AA, AN, AO') #occupations plus continental debt for conneticut
print(occs_and_debt)
no_occ = occs_and_debt[occs_and_debt['Occupation'].isna()]
num_of_people = len(no_occ)
no_occ = no_occ.reset_index(drop=True)
no_occ[['Cents', 'Cents.1', 'Cents.2']] = no_occ[['Cents', 'Cents.1', 'Cents.2']].div(100)
total_series = no_occ[['Dollars', 'Dollars.1', 'Dollars.2', 'Cents', 'Cents.1', 'Cents.2']].sum(axis=1)
total = total_series.sum()

occs_debt_wo_dups = occs_debt_wo_dups.append({'Occupation': 'None Listed', 'Total': total, '# of People': num_of_people, 'Average': total / num_of_people}, ignore_index=True)
print(occs_debt_wo_dups[['Occupation', 'Total', '# of People', 'Average', '% of Total Debt']].to_string())

In [ ]:
occs_debt_wo_dups[['Occupation', 'Total', '# of People', 'Average', '% of Total Debt']].to_excel("./FinishedSpreadsheets/occupation_debt_" + state + ".xlsx")